# Ejercicio 7: Bases de Datos Vectoriales

## Michael Perugachi

## Objetivo de la práctica

Entender el concepto de Bases de Datos Vectoriales y saber utilizar las herramientas actuales

## Parte 0: Carga del Corpus

Vamos a utilizar la API de Kaggle para acceder al dataset _Wikipedia Text Corpus for NLP and LLM Projects_

El corpus está disponible desde este [link](https://www.kaggle.com/datasets/gzdekzlkaya/wikipedia-text-corpus-for-nlp-and-llm-projects?utm_source=chatgpt.com)

### Actividad

1. Carga el corpus


In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

In [ ]:
# Set the path to the file you'd like to load
file_path = "wikipedia_text_corpus.csv"

# Load the latest version
df = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "gzdekzlkaya/wikipedia-text-corpus-for-nlp-and-llm-projects",
  file_path,
)

df.head()

## Parte 1: Generación de Embeddings

Vamos a utilizar E5 como modelo de embeddings.

La documentación de E5 está disponible desde este [link](https://huggingface.co/intfloat/e5-base-v2)

### Actividad

1. Normalizar el corpus
2. Definir una función `chunk_text`, y dividir los textos en _chunks_.
3. Generar embeddings por cada _chunk_

In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import re

df = df.dropna(subset=["text"]).reset_index(drop=True)

# Limpieza básica
def normalize_text(s: str) -> str:
    s = re.sub(r"\s+", " ", s).strip()
    return s

df["text_norm"] = df["text"].astype(str).map(normalize_text)

df.head()

In [ ]:
def chunk_text(text: str, max_chars: int = 800, overlap: int = 100):
    """
    Chunking por caracteres.
    max_chars ~ 600-1000 suele funcionar bien.
    overlap ayuda a no cortar ideas a la mitad.
    """
    chunks = []
    start = 0
    n = len(text)
    while start < n:
        end = min(start + max_chars, n)
        chunk = text[start:end]
        chunk = chunk.strip()
        if len(chunk) > 0:
            chunks.append(chunk)
        if end == n:
            break
        start = max(0, end - overlap)
    return chunks

records = []
for i, row in df.iterrows():
    chunks = chunk_text(row["text_norm"], max_chars=800, overlap=100)
    for j, ch in enumerate(chunks):
        records.append({
            "doc_id": int(i),
            "chunk_id": j,
            "text": ch
        })

chunks_df = pd.DataFrame(records)
chunks_df.head(), len(chunks_df)

In [ ]:
from sentence_transformers import SentenceTransformer

MODEL_NAME = "intfloat/e5-base-v2"   # recomendado para retrieval
model = SentenceTransformer(MODEL_NAME)

# Textos a indexar (pasajes)
passages = ["passage: " + t for t in chunks_df["text"].tolist()]

In [ ]:
# Embeddings (N x D)
# Se debe usar normalize_embeddings=True para similitud coseno
embeddings = model.encode(
    passages,
    batch_size=16,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
).astype("float32")

In [ ]:
print(embeddings.shape, embeddings.dtype)

In [ ]:
def embed_query(query: str) -> np.ndarray:
    q = "query: " + query
    vec = model.encode(
        [q],
        convert_to_numpy=True,
        normalize_embeddings=True
    ).astype("float32")
    return vec

query_text = "Battery measuring"

query_vec = embed_query(query_text)
query_vec.shape

## Parte 2: FAISS

FAISS es una librería para búsqueda por similitud eficiente y clustering de vectores densos.

La documentación de FAISS está disponible en este [link](https://faiss.ai/index.html)

### Actividad

1. Crea un índice en FAISS
2. Carga los embeddings
3. Realiza una búsqueda a partir de una _query_

In [ ]:
!pip install faiss-cpu
import faiss

# Dimensión de los embeddings
dim = embeddings.shape[1]

# Crear índice FAISS
index = faiss.IndexFlatIP(dim)

In [ ]:
# Agregar los embeddings al índice
index.add(embeddings)

# Verificar cantidad de vectores indexados
print("Vectores indexados:", index.ntotal)


In [ ]:
# Número de resultados a recuperar
k = 5

# Búsqueda en el índice
distances, indices = index.search(query_vec, k)
for i, idx in enumerate(indices[0]):
    # Add a check to ensure idx is within valid bounds
    if 0 <= idx < len(passages):
        print(f"\nResultado {i + 1}")
        print(f"Score (similitud): {distances[0][i]:.4f}")
        print(f"Texto:\n{passages[idx][:400]}...")
    else:
        print(f"\nResultado {i + 1} (Índice inválido: {idx})")
        print(f"Score (similitud): {distances[0][i]:.4f}")
        print("Texto: <No disponible debido a índice fuera de rango>")

## Parte 3 — Vector DB #1: Qdrant (búsqueda vectorial + metadata)

### Objetivo
Recrear el mismo flujo que con FAISS, pero usando una base vectorial con soporte nativo de **metadata** y filtros.

### Qué debes implementar
1. Levantar / conectar con una instancia de Qdrant.
2. Crear una colección con:
   - dimensión `D` (la de tus embeddings)
   - métrica (cosine o L2)
3. Insertar:
   - `id`
   - `embedding`
   - `payload` (metadata: texto, título, etiquetas, etc.)
4. Consultar Top-k por similitud:
   - `query_embedding`
   - `k`

### Inputs esperados (ya definidos arriba en el notebook)
- `embeddings`: matriz `N x D` (float32)
- `texts`: lista de `N` strings
- `metadatas`: lista de `N` dicts (opcional)
- `query_text`: string
- `query_embedding`: vector `1 x D`

### Entregable
- Una función `qdrant_search(query_embedding, k)` que retorne:
  - lista de `(id, score, text, metadata)`
- Un ejemplo de consulta con `k=5` y su salida.

### Preguntas
**- ¿La métrica usada fue cosine o L2? ¿Por qué?**
Se utilizo la metrica cosine ya que este permite medir que tan alineados estan dos vetores sin verse afectada por su magnitud. Esto es ideal para la busqueda semantica

**- ¿Qué tan fácil fue filtrar por metadata en comparación con FAISS?**
Fue mas facil, ya que en FAISS no existen filtros nativos, por lo que filtrar metadata requiere codigo adicional y estructuras externas.
En Qdrant el filtrado por metadata es nativo y mas sencillo, ya que cada vector tiene un payload JSON y el motor permite consultas filtradas directamente

**- ¿Qué pasa con el tiempo de respuesta cuando aumentas `k`?**
Al aumentar k, el tiempo de respuesta tambien aumenta ligeramente, porque el motor debe devolver y ordenar mas resultados.


In [ ]:
!pip install -q qdrant-client


In [ ]:
import numpy as np
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct


In [ ]:
qdrant_client = QdrantClient(":memory:")  # Correcto para Colab
collection_name = "documents"


In [ ]:
dim = embeddings.shape[1]

# Borrar colección si ya existe (evita errores)
try:
    qdrant_client.delete_collection(collection_name)
except:
    pass

qdrant_client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(
        size=dim,
        distance=Distance.COSINE
    )
)


In [ ]:
points = []

for i, emb in enumerate(embeddings):
    payload = {
        "text": passages[i]
    }

    if 'metadatas' in globals() and metadatas is not None:
        payload.update(metadatas[i])

    points.append(
        PointStruct(
            id=i,
            vector=emb.tolist(),   #  vector 1D
            payload=payload
        )
    )

qdrant_client.upsert(
    collection_name=collection_name,
    points=points
)


In [ ]:
def qdrant_search(query_embedding: np.ndarray, k: int = 5):
    """
    Retorna: (id, score, text, metadata)
    Compatible con Qdrant en Google Colab
    """

    # Asegurar vector 1D
    if query_embedding.ndim == 2:
        query_embedding = query_embedding[0]

    search_result = qdrant_client.query_points(
        collection_name=collection_name,
        query=query_embedding.tolist(),
        limit=k,
        with_payload=True
    )

    results = []
    for hit in search_result.points:
        results.append((
            hit.id,
            hit.score,
            hit.payload.get("text"),
            hit.payload
        ))

    return results


In [ ]:
k = 5
results = qdrant_search(query_vec, k)

for i, (doc_id, score, text, metadata) in enumerate(results):
    print(f"\nResultado {i + 1}")
    print(f"ID: {doc_id}")
    print(f"Score (similitud): {score:.4f}")
    print(f"Texto:\n{text[:400]}...")


## Parte 4 — Vector DB #2: Milvus (indexación ANN y escalabilidad)

### Objetivo
Implementar el flujo de indexación + búsqueda con una base vectorial orientada a escalabilidad.

### Qué debes implementar
1. Conectar a Milvus.
2. Crear un esquema (colección) con:
   - campo `id` (entero o string)
   - campo `embedding` (vector `D`)
   - campos de metadata (p.ej., `category`, `source`, `title`)
3. Insertar `N` embeddings.
4. Crear/seleccionar un índice ANN (ej. HNSW o IVF).
5. Ejecutar consultas Top-k y recuperar textos asociados.

### Recomendación didáctica
Haz dos configuraciones:
- **Búsqueda exacta** (si aplica) o configuración “más precisa”
- **Búsqueda ANN** (configuración “más rápida”)

Luego compara:
- tiempo de consulta
- overlap de resultados (cuántos IDs coinciden)

### Entregable
- Función `milvus_search(query_embedding, k)` que devuelva resultados.
- Un mini experimento: `k=5` y `k=20` (tiempos y resultados).

### Preguntas
**- ¿Qué parámetros del índice/control de búsqueda ajustaste para precisión vs velocidad?**
Se ajustaron dos parametros, primero el tipo de indice "IVF_FLAT" con "nlist=128", que habilita la busqueda aproximada(ANN)
El segundo parametro fue el de busqueda "nprobe" a un valor de 1 paera que la consulta se realice a un solo cluster haciendolo mas rapido.

**- ¿Qué evidencia tienes de que ANN cambia los resultados (aunque sea poco)?**
En el codigo se calcula el porcentaje de coincidencia "overlap" entre busquedas. Como el overlap es menor al 100%, se demuestra que ANN devuelve resultados diferentes.


In [ ]:
!pip install -q "pymilvus[milvus_lite]"


In [ ]:
from pymilvus import MilvusClient, DataType
import numpy as np
import time

# URI SIMPLE, SIN VARIABLES RARAS
client = MilvusClient(uri="milvus_demo.db")

print("Milvus Lite iniciado correctamente")


In [ ]:
collection_name = "documents_milvus"
dim = embeddings.shape[1]

if client.has_collection(collection_name):
    client.drop_collection(collection_name)

schema = MilvusClient.create_schema(
    auto_id=False,
    enable_dynamic_field=True
)

schema.add_field("id", DataType.INT64, is_primary=True)
schema.add_field("embedding", DataType.FLOAT_VECTOR, dim=dim)
schema.add_field("text", DataType.VARCHAR, max_length=2048)
schema.add_field("category", DataType.VARCHAR, max_length=50)

index_params = client.prepare_index_params()
index_params.add_index(
    field_name="embedding",
    index_type="IVF_FLAT",
    metric_type="COSINE",
    params={"nlist": 128}
)

client.create_collection(
    collection_name=collection_name,
    schema=schema,
    index_params=index_params
)


In [ ]:
batch_size = 500  # seguro para Colab

for start in range(0, len(embeddings), batch_size):
    end = start + batch_size

    batch_data = [
        {
            "id": i,
            "embedding": embeddings[i].tolist(),
            "text": passages[i],
            "category": "documento"
        }
        for i in range(start, min(end, len(embeddings)))
    ]

    client.insert(
        collection_name=collection_name,
        data=batch_data
    )

    print(f"Insertados documentos {start} → {end}")


In [ ]:
import time

def milvus_search(query_embedding, k, nprobe):
    """
    Búsqueda en Milvus Lite
    """
    search_params = {
        "metric_type": "COSINE",
        "params": {"nprobe": nprobe}
    }

    start = time.time()
    results = client.search(
        collection_name=collection_name,
        data=[query_embedding.tolist()],
        limit=k,
        search_params=search_params,
        output_fields=["text", "category"]
    )
    elapsed = (time.time() - start) * 1000

    hits = []
    for hit in results[0]:
        hits.append((
            hit["id"],
            hit["distance"],
            hit["entity"]["text"],
            hit["entity"]
        ))

    return hits, elapsed


In [ ]:
query_vec = query_vec.squeeze()

for k in [5, 20]:
    print(f"\n🔹 k = {k}")

    # Más precisa
    res_precise, t_precise = milvus_search(query_vec, k, nprobe=128)

    # Más rápida (ANN)
    res_fast, t_fast = milvus_search(query_vec, k, nprobe=1)

    ids_precise = {r[0] for r in res_precise}
    ids_fast = {r[0] for r in res_fast}

    overlap = len(ids_precise & ids_fast) / k * 100

    print(f"Precisa: {t_precise:.2f} ms")
    print(f"Rápida:  {t_fast:.2f} ms")
    print(f"Overlap: {overlap:.1f}%")
    print(f"Ejemplo texto: {res_fast[0][2][:100]}...")


## Parte 5 — Vector DB #3: Weaviate (búsqueda semántica con esquema)

### Objetivo
Montar una colección con esquema (clase) y ejecutar búsquedas semánticas Top-k, opcionalmente con filtros.

### Qué debes implementar
1. Conectar a Weaviate.
2. Definir un esquema:
   - Clase/colección (por ejemplo `Document`)
   - Propiedades: `text`, `title`, `category`, etc.
   - Vector asociado (embedding)
3. Insertar objetos con:
   - propiedades + vector
4. Consultar por similitud (Top-k) con `query_embedding`.
5. (Opcional) agregar un filtro por propiedad (metadata).

### Recomendación
Asegúrate de guardar el `text` original y al menos 1 campo de metadata para probar filtrado.

### Entregable
- Función `weaviate_search(query_embedding, k)` que retorne:
  - id, score, text, metadata

### Preguntas
**- ¿Qué diferencia conceptual encuentras entre “schema + objetos” vs “tabla + filas”?**
El modelo "schema + objetos" permite que cada objeto almacene tanto sus datos como su embedding y metadata, integrando directamente la busqueda semantica.
El modelo "tabla + filas" de una base relacional solo maneja datos estructurados sin representacion semantica nativa, por lo que la busqueda se limita a coincidencias exactas.

**- ¿Cómo describirías el trade-off de complejidad vs expresividad?**

Weaviate es mas expresivo porque soporta IA y busqueda vetorial de forma nativa, pero esto tambien lo hace mas complejo de diseñar y operar. Las bases relacionales son mas simples y conocidas, aunque menos adecuadas para tareas semanticas.

In [ ]:
!pip install weaviate-client


In [ ]:
import weaviate
from weaviate.embedded import EmbeddedOptions
import time


In [ ]:
import weaviate
from weaviate.embedded import EmbeddedOptions
import time

# The embedded_options argument is not directly used in connect_to_embedded() in this client version.
# Calling it without arguments will start an embedded instance with default settings.
client = weaviate.connect_to_embedded()

print("Weaviate iniciado correctamente")

In [ ]:
collection_name = "Document"

# Si existe, eliminarla
if client.collections.exists(collection_name):
    client.collections.delete(collection_name)

# Crear colección
collection = client.collections.create(
    name=collection_name,
    vectorizer_config=None,  # embeddings externos
    properties=[
        weaviate.classes.config.Property(
            name="text",
            data_type=weaviate.classes.config.DataType.TEXT
        ),
        weaviate.classes.config.Property(
            name="category",
            data_type=weaviate.classes.config.DataType.TEXT
        )
    ]
)

print("Colección creada correctamente")


In [ ]:
batch_size = 200

with collection.batch.dynamic() as batch:
    for i in range(len(embeddings)):
        batch.add_object(
            properties={
                "text": passages[i],
                "category": "documento"
            },
            vector=embeddings[i].tolist()
        )

print("Documentos insertados en Weaviate")


In [ ]:
def weaviate_search(query_embedding, k):
    start = time.time()

    results = collection.query.near_vector(
        near_vector=query_embedding.tolist(),
        limit=k,
        return_metadata=["distance"]
    )

    elapsed = (time.time() - start) * 1000

    hits = []
    for obj in results.objects:
        hits.append({
            "id": obj.uuid,
            "score": 1 - obj.metadata.distance,
            "text": obj.properties["text"],
            "metadata": {
                "category": obj.properties["category"]
            }
        })

    return hits, elapsed


In [ ]:
query_embedding = embeddings[0]

for k in [5, 20]:
    results, time_ms = weaviate_search(query_embedding, k)

    print(f"\n🔹 k={k}")
    print(f"Tiempo: {time_ms:.2f} ms")
    print(f"Ejemplo texto:\n{results[0]['text'][:150]}...")


## Parte 6 — Vector Store #4: Chroma (prototipado rápido)

### Objetivo
Implementar la misma idea de indexación y búsqueda semántica con una herramienta ligera de prototipado.

### Qué debes implementar
1. Crear una colección.
2. Insertar:
   - ids
   - embeddings
   - documents (texto)
   - metadatas (opcional)
3. Consultar Top-k con `query_embedding`.

### Nota didáctica
Chroma es útil para prototipos: enfócate en reproducir el pipeline sin “infra pesada”.

### Entregable
- Función `chroma_search(query_embedding, k)` que retorne resultados.
- Una consulta con `k=5`.

### Preguntas
**- ¿Qué tan fácil fue implementar todo comparado con Qdrant/Milvus?**
ChromaDB fue mas facil de implementar que Qdrant y Milvus, porque no requiere definir esquemas, indices ni parametros avanzados. Solo se crea la coleccion y se insertan los embeddings directamente.

**- ¿Qué limitaciones ves para un sistema en producción?**

ChromaDB esta mas orientado a prototipos locales. Tiene menos soporte para escalabilidad, alta disponibilidad, control avanzado de indices y rendimiento en grandes volumenes de datos, por lo que no es tan robusto como Qdrant o Milvus para sistemas de produccion de gran escala.

In [ ]:
!pip install -q chromadb


In [ ]:
import chromadb
from chromadb.config import Settings
import time

chroma_client = chromadb.Client()
collection = chroma_client.get_or_create_collection(
    name="documents"
)

In [ ]:
# Define ids, documents, and metadatas from previously generated data
ids = [str(i) for i in range(len(embeddings))]
documents = passages  # Using 'passages' which contains the chunked texts
# Create metadatas from chunks_df, ensuring alignment with embeddings/passages
metadatas = chunks_df[['doc_id', 'chunk_id']].to_dict(orient='records')

# Split into smaller batches as ChromaDB has a batch size limit
batch_size = 5000  # A batch size smaller than 5461

for i in range(0, len(ids), batch_size):
    batch_ids = ids[i:i + batch_size]
    batch_documents = documents[i:i + batch_size]
    batch_embeddings = embeddings[i:i + batch_size]
    batch_metadatas = metadatas[i:i + batch_size]

    collection.add(
        ids=batch_ids,
        documents=batch_documents,
        embeddings=batch_embeddings,
        metadatas=batch_metadatas
    )
    print(f"Inserted batch {i//batch_size + 1} of {len(ids)//batch_size + 1}")



In [ ]:
def chroma_search(query_embedding, k=5):
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=k,
        include=["documents", "metadatas", "distances"]
    )

    output = []
    for i in range(len(results["ids"][0])):
        output.append({
            "id": results["ids"][0][i],
            "score": results["distances"][0][i],
            "text": results["documents"][0][i],
            "metadata": results["metadatas"][0][i] if results["metadatas"] else None
        })

    return output


In [ ]:
results = chroma_search(query_embedding, k=5)

for r in results:
    print(r)


## Parte 7 — SQL + vectores: PostgreSQL/pgvector (vector search transparente)

### Objetivo
Guardar embeddings en una tabla y ejecutar una consulta SQL de similitud.

### Qué debes implementar
1. Conectar a una base PostgreSQL con `pgvector` habilitado.
2. Crear una tabla (ej. `documents`) con:
   - `id` (PK)
   - `text` (texto)
   - `embedding` (vector(D))
   - metadata (columnas adicionales)
3. Insertar todos los documentos y embeddings.
4. Consultar Top-k por similitud, ordenando por distancia.

### Fórmula conceptual (lo que implementa tu SQL)
Para una consulta `q`, buscas:
$$ argmin_d \in D \; \text{dist}(\vec{q}, \vec{d})$$
donde `dist` puede ser L2 o una variante para cosine (según configuración).

### Entregable
- Función `pgvector_search(query_embedding, k)` que ejecute SQL y devuelva:
  - id, score/distancia, text, metadata

### Preguntas
**- ¿Qué tan “explicable” te parece esta aproximación vs las otras?**
SQL es mas explicable y transparente porque ves claramente como se calcula la similitud y como se ordenan los resultados.

**- ¿Qué ventajas ofrece el mundo SQL (JOIN, filtros, agregaciones)?**
SQL permite JOINs, filtros complejos y agregaciones nativas, lo que facilita combinar embeddings con datos estructurados. Esto hace que construir analisis y reportes sea muy flexible sin salirte del motor SQL.

**- ¿Qué limitaciones esperas en escalabilidad frente a bases vectoriales dedicadas?**
SQL no escala tan bien como Qdrant o Milvus para millones de vetores, porque normalmente usa busqueda exacta y no indices ANN optimizados. Esto aumenta la latencia y el consumo de recursos cuando crece el volumen de datos o las consultas.

In [ ]:
pip install psycopg2-binary numpy

In [ ]:
import duckdb
import numpy as np
import json

# --------------------------------------------------
# 1. Configuración
# --------------------------------------------------
DIMENSION = len(embeddings[0])
con = duckdb.connect(":memory:")

# --------------------------------------------------
# 2. Crear tabla
# --------------------------------------------------
con.sql(f"""
CREATE TABLE documents (
    id INTEGER PRIMARY KEY,
    text VARCHAR,
    embedding FLOAT[{DIMENSION}],
    metadata JSON
)
""")

# --------------------------------------------------
# 3. Insertar documentos + embeddings
# --------------------------------------------------
data_to_insert = []

for i in range(len(documents)):
    vec = embeddings[i].tolist()
    meta = json.dumps(metadatas[i]) if isinstance(metadatas[i], dict) else json.dumps({})
    txt = documents[i]
    data_to_insert.append((i, txt, vec, meta))

con.executemany(
    "INSERT INTO documents VALUES (?, ?, ?, ?)",
    data_to_insert
)

print(f"Documentos insertados: {len(data_to_insert)}")

# --------------------------------------------------
# 4. Función entregable: pgvector_search
# --------------------------------------------------
def pgvector_search(query_embedding, k=5):
    """
    Ejecuta búsqueda semántica Top-k usando SQL + vectores.
    Retorna: id, score/distancia, text, metadata
    """
    q_list = query_embedding.tolist() if hasattr(query_embedding, "tolist") else query_embedding

    results = con.sql(f"""
        SELECT
            id,
            (1.0 - list_cosine_similarity(embedding, {q_list}::FLOAT[{DIMENSION}])) AS distance,
            text,
            metadata
        FROM documents
        ORDER BY distance ASC
        LIMIT {k}
    """).fetchall()

    response = []
    for r in results:
        response.append({
            "id": r[0],
            "score": r[1],          # distancia (menor = más similar)
            "text": r[2],
            "metadata": json.loads(r[3])
        })

    return response

# --------------------------------------------------
# 5. Consulta de prueba
# --------------------------------------------------
query_embedding = embeddings[0]  # o uno nuevo generado
results = pgvector_search(query_embedding, k=5)

for r in results:
    print(r)


In [ ]:
from google.colab import output
output.clear()